In [2]:
!pip install -r ../../requirements.txt

  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl (14 kB)
  Using cached aiohttp-3.10.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:02m
  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Using cached charset_normalizer-3.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 1.5 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 KB 1.8 MB/s eta 0:00:000:00:01m eta 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 KB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━

In [3]:
import random
import time
import os

from IPython.display import clear_output

import torch
from torch import nn, optim, cuda
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchmetrics
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
BS = 32

ModuleNotFoundError: No module named 'torch'

In [ ]:
device = 'cpu'
if cuda.is_available():
    device_name = cuda.get_device_name(0)
    memory = cuda.get_device_properties(0).total_memory / (1024 ** 3)
    print(f"Device {device_name} has {memory} GB of memory")
    device = torch.device('cuda')
else:
    raise Exception("Cuda isn't available")
device

In [ ]:
train_transform = transforms.Compose([
    # transforms.ConvertImageDtype(dtype=torch.float32),
    transforms.ToTensor(),
    transforms.Resize([224, 224], antialias=False),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.RandomResizedCrop(224, antialias=False),
])


val_transform = transforms.Compose([
    # transforms.ConvertImageDtype(dtype=torch.float32),
    transforms.ToTensor(),
    transforms.Resize([224, 224], antialias=False),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = datasets.ImageFolder(root='./dataset/tr', transform=train_transform)
val_dataset = datasets.ImageFolder(root='./dataset/val', transform=val_transform)
test_dataset = datasets.ImageFolder(root='./dataset/test', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False)

In [ ]:
from collections import Counter

train_labels = train_dataset.targets
val_labels = val_dataset.targets
test_labels = test_dataset.targets

class_counts_train = Counter(train_labels)
class_counts_val = Counter(val_labels)
class_counts_test = Counter(test_labels)

print("Train Dataset Class Counts:")
for class_index, count in class_counts_train.items():
    class_name = train_dataset.classes[class_index]
    print(f"{class_name}: {count}")

print("Val Dataset Class Counts:")
for class_index, count in class_counts_val.items():
    class_name = val_dataset.classes[class_index]
    print(f"{class_name}: {count}")

print("Test Dataset Class Counts:")
for class_index, count in class_counts_test.items():
    class_name = test_dataset.classes[class_index]
    print(f"{class_name}: {count}")

In [ ]:
try:
  if model:
    model.zero_grad()
    del model
    torch.cuda.empty_cache()
    print('model has been deleted')
except:
  print('model doesnt exist')